In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!ls

dataset  drive	sample_data


In [ ]:
!pwd

/content


**extract zip file**

In [ ]:
!unzip "drive/My Drive/IBM_Dataset/dataset.zip" -d "drive/My Drive/Sprint2/"


Archive:  drive/My Drive/IBM_Dataset/dataset.zip
  inflating: drive/My Drive/Sprint2/dataset/readme.txt  
   creating: drive/My Drive/Sprint2/dataset/test_set/
   creating: drive/My Drive/Sprint2/dataset/test_set/Cyclone/
  inflating: drive/My Drive/Sprint2/dataset/test_set/Cyclone/867.jpg  
  inflating: drive/My Drive/Sprint2/dataset/test_set/Cyclone/868.jpg  
  inflating: drive/My Drive/Sprint2/dataset/test_set/Cyclone/869.jpg  
  inflating: drive/My Drive/Sprint2/dataset/test_set/Cyclone/870.jpg  
  inflating: drive/My Drive/Sprint2/dataset/test_set/Cyclone/871.jpg  
  inflating: drive/My Drive/Sprint2/dataset/test_set/Cyclone/872.jpg  
  inflating: drive/My Drive/Sprint2/dataset/test_set/Cyclone/873.jpg  
  inflating: drive/My Drive/Sprint2/dataset/test_set/Cyclone/874.jpg  
  inflating: drive/My Drive/Sprint2/dataset/test_set/Cyclone/875.jpg  
  inflating: drive/My Drive/Sprint2/dataset/test_set/Cyclone/876.jpg  
  inflating: drive/My Drive/Sprint2/dataset/test_set/Cyclone/877.jpg

**importing image data generator library**

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

**Image Data Augmentation**

In [ ]:
#Configuring image Data Generator Class

#Image Augmentation for training data

train_datagen=ImageDataGenerator(rescale=1./255,zoom_range=0.2,horizontal_flip=True,shear_range=0.2)

In [ ]:
#Image Data Augmentation for testing data

test_datagen=ImageDataGenerator(rescale=1./255)

In [ ]:
#Performing data augmentation to train data

x_train=train_datagen.flow_from_directory(r"/content/drive/MyDrive/Sprint2/dataset/train_set",target_size=(64,64),
                                                            batch_size=5,color_mode='rgb',class_mode='categorical')

Found 742 images belonging to 4 classes.


In [ ]:
#performing data augmentation to test data

x_test=test_datagen.flow_from_directory(r"/content/drive/MyDrive/Sprint2/dataset/test_set",target_size=(64,64),
                                                            batch_size=5,color_mode='rgb',class_mode="categorical")

Found 198 images belonging to 4 classes.


**importing neccessary libraries**

In [ ]:
import numpy as np
import tensorflow
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Conv2D,MaxPooling2D,Flatten

In [ ]:
# initialising the model and adding CNN layers

model=Sequential()

# First convolution layer and pooling

model.add(Conv2D(32,(3,3),activation="relu",input_shape=(64,64,3)))
model.add(MaxPooling2D(pool_size=(2,2)))

#Second convolution layer and pooling

model.add(Conv2D(32,(3,3),activation="relu"))
model.add(MaxPooling2D(pool_size=(2,2)))

#Flattening the layers

model.add(Flatten())

#Adding Dense Layers
model.add(Dense(units=128,activation='relu'))
model.add(Dense(units=4,activation='softmax'))

model.compile(loss="categorical_crossentropy",metrics=["accuracy"],optimizer='adam')

In [ ]:
# Summary of our model

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 62, 62, 32)        896       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 31, 31, 32)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 29, 29, 32)        9248      
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 14, 14, 32)       0         
 2D)                                                             
                                                                 
 flatten (Flatten)           (None, 6272)              0         
                                                                 
 dense (Dense)               (None, 128)               8

In [ ]:
# Fitting the model

model.fit_generator(generator=x_train,epochs=20,steps_per_epoch=len(x_train),validation_data=x_test,validation_steps=len(x_test))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  """Entry point for launching an IPython kernel.


Epoch 1/20
149/149 [==============================] - 37s 244ms/step - loss: 1.1542 - accuracy: 0.5040 - val_loss: 0.9826 - val_accuracy: 0.5859
Epoch 2/20
149/149 [==============================] - 31s 208ms/step - loss: 0.8935 - accuracy: 0.6199 - val_loss: 0.9013 - val_accuracy: 0.6111
Epoch 3/20
149/149 [==============================] - 30s 204ms/step - loss: 0.7845 - accuracy: 0.6981 - val_loss: 0.7103 - val_accuracy: 0.6919
Epoch 4/20
149/149 [==============================] - 36s 240ms/step - loss: 0.6674 - accuracy: 0.7480 - val_loss: 0.7855 - val_accuracy: 0.7626
Epoch 5/20
149/149 [==============================] - 31s 211ms/step - loss: 0.5963 - accuracy: 0.7736 - val_loss: 0.8185 - val_accuracy: 0.6970
Epoch 6/20
149/149 [==============================] - 31s 210ms/step - loss: 0.5871 - accuracy: 0.7749 - val_loss: 0.7057 - val_accuracy: 0.7273
Epoch 7/20
149/149 [==============================] - 32s 215ms/step - loss: 0.4730 - accuracy: 0.8167 - val_loss: 0.5963 - val_ac

In [ ]:
# Save the model

model.save('disaster.h5')
model_json=model.to_json()
with open("model-bw.json","w") as json_file:
  json_file.write(model_json)

In [ ]:
# Load the saved model

from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image
model=load_model('disaster.h5')

In [ ]:
x_train.class_indices

{'Cyclone': 0, 'Earthquake': 1, 'Flood': 2, 'Wildfire': 3}

In [ ]:
# taking image as input

img=image.load_img(r"/content/drive/MyDrive/Sprint2/dataset/test_set/Earthquake/1329.jpg",target_size=(64,64))
x=image.img_to_array(img)
x=np.expand_dims(x,axis=0)
index=['Cyclone','Earthquake','Flood','Wildfire']
y=np.argmax(model.predict(x),axis=1)
print(index[int(y)])

1/1 [==============================] - 0s 96ms/step
Flood


In [ ]:
# taking image as input

img=image.load_img(r"/content/drive/MyDrive/Sprint2/dataset/test_set/Cyclone/900.jpg",target_size=(64,64))
x=image.img_to_array(img)
x=np.expand_dims(x,axis=0)
index=['Cyclone','Earthquake','Flood','Wildfire']
y=np.argmax(model.predict(x),axis=1)
print(index[int(y)])

1/1 [==============================] - 0s 21ms/step
Cyclone
